In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sys
#reload(sys)
#sys.setdefaultencoding("utf-8")
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
# let's read them directly from mysql
engine = create_engine(# insert connection here)
# Load all the bad reviews
bad_reviews = pd.read_sql_query('SELECT R.*, B.business_name, B.state, B.postal_code FROM reviews as R LEFT JOIN businesses as B ON R.business_id=B.business_id WHERE B.categories LIKE "%%Restaurant%%" AND R.stars <= 2 LIMIT 50000;', engine)

In [15]:
# Load all the bad reviews
good_reviews = pd.read_sql_query('SELECT R.*, B.business_name, B.state, B.postal_code FROM reviews as R LEFT JOIN businesses as B ON R.business_id=B.business_id WHERE B.categories LIKE "%%Restaurant%%" AND R.stars >= 4 LIMIT 50000;', engine)

In [21]:
good_reviews.head()

,business_id,cool,review_date,funny,review_id,stars,review_text,review_type,useful,user_id,business_name,state,postal_code,category
0,RESDUcs7fIiihp38-d6_6g,0,2016-02-14,0,--1h6ROOo2XEsLQK8uKvaQ,4,Mind you the wait time could be loooong! So br...,review,0,2rMCI2OU7ZIusbjnXGomdQ,Bacchanal Buffet,NV,89109.0,good
1,Pm6LSrQySKyLS5b_a6CGSA,0,2015-07-16,0,--CKvYCwEJLH7bsNWR2H2A,5,The service was excellent ! Owner was very fri...,review,0,48QPXaCpdTitl4QgPs7Ddg,Panini Bread & Grill,AZ,85308.0,good
2,LH-DYCy9v3kM4JRtpcquxw,0,2010-01-17,0,--eoBFx6uE0Qfcv-WuS8OA,5,I recently dined at the Silk Road during a rec...,review,0,66kcBWJP6Km3K-nvTGC-1Q,Silk Road,NV,89109.0,good
3,qmymSqVwHYRqdwfcBatzpQ,1,2016-02-20,1,--FbRLO-6R3fB4YNlAlomA,5,This is one of the best restaurant in Summerli...,review,1,9wyWvteLBBHPeqs52QZ-hg,Vintner Grill,NV,89135.0,good
4,_nB8HNT2IDRhlAoY9TPYew,0,2014-04-24,0,--FwX6sSeRtPFSeenaiLiQ,5,I love this place. Get take out a couple times...,review,0,93BMxByxerhAvG1ccUUQPA,Sushi Time,AZ,85282.0,good


# Scattertext Experimentation

In [24]:
import scattertext as st
import spacy
from pprint import pprint

In [3]:
convention_df = st.SampleCorpora.ConventionData2012.get_data()

In [4]:
convention_df.head()

,party,speaker,text
0,democrat,BARACK OBAMA,Thank you. Thank you. Thank you. Thank you so ...
1,democrat,MICHELLE OBAMA,"Thank you so much. Tonight, I am so thrilled a..."
2,democrat,RICHARD DURBIN,Thank you. It is a singular honor to be here t...
3,democrat,JOSEPH BIDEN,"Hey, Delaware. \nAnd my favorite Democrat, Jil..."
4,democrat,JILL BIDEN,"Hello. \nThank you, Angie. I'm so proud of how..."


Ok. In my case, let's combine the bad reviews with the good ones. Insert a categorical variable that labels the review as being either "bad" or "good" for sake of simplicity.

In [17]:
temp = bad_reviews
temp['general'] = 'bad'

In [61]:
bad_reviews['category'] = 'bad'
good_reviews['category'] = 'good'

In [26]:
all_reviews = bad_reviews.append(good_reviews, ignore_index=True)

NameError: name 'bad_reviews' is not defined

In [63]:
print all_reviews.shape
print all_reviews.head()

In [64]:
# Decode the review_text column

all_reviews.loc[:,'review_text'][10]
# all_reviews['review_text'] = all_reviews['review_text'].apply( lambda x: x.decode())

u"Food was great!\n\nLove the ambience.\n\nMusic is a little too loud... especially if we can't hear each other while sitting next to each other.\n\nA little on the pricey side.\n\nService. s u c k e d! Once food was delivered, server never came to check on us again. Had to wave down someone for our check. Perhaps the server was having an off day?"

In [25]:
# Turn it into a scatterplot corpus
nlp = spacy.en.English()
corpus = st.CorpusFromPandas(all_reviews,
                            category_col = 'category',
                            text_col = 'review_text',
                            nlp = nlp).build()

NameError: name 'all_reviews' is not defined

# That took forever. Let's make the sample size even smaller. 1000 reviews each.

In [2]:
engine = create_engine(# insert connection here)
# Load all the bad reviews
sample_bad = pd.read_sql_query('SELECT R.*, B.business_name, B.state, B.postal_code FROM reviews as R LEFT JOIN businesses as B ON R.business_id=B.business_id WHERE B.categories LIKE "%%Restaurant%%" AND R.stars <= 2 LIMIT 1000;', engine)
sample_good = pd.read_sql_query('SELECT R.*, B.business_name, B.state, B.postal_code FROM reviews as R LEFT JOIN businesses as B ON R.business_id=B.business_id WHERE B.categories LIKE "%%Restaurant%%" AND R.stars >= 4 LIMIT 1000;', engine)

In [3]:
type(sample_bad)
#print sample_good.shape

pandas.core.frame.DataFrame

In [4]:
sample_bad['category'] = 'bad'
sample_good['category'] = 'good'

In [5]:
sample_good.head()

,business_id,cool,review_date,funny,review_id,stars,review_text,review_type,useful,user_id,business_name,state,postal_code,category
0,RESDUcs7fIiihp38-d6_6g,0,2016-02-14,0,--1h6ROOo2XEsLQK8uKvaQ,4,Mind you the wait time could be loooong! So br...,review,0,2rMCI2OU7ZIusbjnXGomdQ,Bacchanal Buffet,NV,89109,good
1,Pm6LSrQySKyLS5b_a6CGSA,0,2015-07-16,0,--CKvYCwEJLH7bsNWR2H2A,5,The service was excellent ! Owner was very fri...,review,0,48QPXaCpdTitl4QgPs7Ddg,Panini Bread & Grill,AZ,85308,good
2,LH-DYCy9v3kM4JRtpcquxw,0,2010-01-17,0,--eoBFx6uE0Qfcv-WuS8OA,5,I recently dined at the Silk Road during a rec...,review,0,66kcBWJP6Km3K-nvTGC-1Q,Silk Road,NV,89109,good
3,qmymSqVwHYRqdwfcBatzpQ,1,2016-02-20,1,--FbRLO-6R3fB4YNlAlomA,5,This is one of the best restaurant in Summerli...,review,1,9wyWvteLBBHPeqs52QZ-hg,Vintner Grill,NV,89135,good
4,_nB8HNT2IDRhlAoY9TPYew,0,2014-04-24,0,--FwX6sSeRtPFSeenaiLiQ,5,I love this place. Get take out a couple times...,review,0,93BMxByxerhAvG1ccUUQPA,Sushi Time,AZ,85282,good


In [6]:
sample_all = sample_bad.append(sample_good, ignore_index=True)

In [7]:
sample_all['review_text'] = sample_all['review_text'].apply(lambda x: x.decode())

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 58: ordinal not in range(128)

In [8]:
sample_all.loc[:,'review_text'][10]

"Food was great!\n\nLove the ambience.\n\nMusic is a little too loud... especially if we can't hear each other while sitting next to each other.\n\nA little on the pricey side.\n\nService. s u c k e d! Once food was delivered, server never came to check on us again. Had to wave down someone for our check. Perhaps the server was having an off day?"

In [12]:
print sample_all.shape

In [14]:
nlp = spacy.en.English()
corpus = st.CorpusFromPandas(sample_all,
                            category_col = 'category',
                            text_col = 'review_text',
                            nlp = nlp).build()

In [15]:
print(list(corpus.get_scaled_f_scores_vs_background().index[:10]))

In [ ]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df['Democratic Score'] = corpus.get_scaled_f_scores('democrat')

In [18]:
html = st.produce_scattertext_explorer(corpus,
                                       category='bad',
                                       category_name='Bad Reviews',
                                       not_category_name='Good Reviews',
                                       width_in_pixels=1000,
                                       metadata = sample_all['stars'])

In [19]:
open("Review-Visualization.html", 'wb').write(html.encode('utf-8'))

# Sentiment Analysis EDA

In [9]:
from textblob import TextBlob

In [10]:
sample_all.loc[:, 'review_text'][100]

"The dunes, there's a lot to say about this place. It's not great by any means. The staff for the most part is always pretty good. They're is a high turn around on the bar tenders however. That's probably because this place is always dead.  The outside of the dunes is God awful and the owners need to spend some money to renovate this dump. Both out side and inside need work. But off the outside of the building I nearly skiped this place. It's looks like a ran down ghetto nudie bar, that you're most likely going to catch something incurable. The only two reasons I go back is because it's so close to my place and beer is cheap. Even the cheap beer prices and really close proximity to my house still make it hard to stop in some days. If it was not for bartender's like Karin and Summer, and the DJ Jeromy who is not always here, and cheap beer. Im sure this place would go under. \n\nThe owners of the dunes need to get off there ass. Recognize that your bartender's work to save this dump, th

In [11]:
# Create our first TextBlob
dunes = TextBlob(str(sample_all.loc[:, 'review_text'][100]))

In [12]:
# Part-of-speech tagging
dunes.tags

[('The', u'DT'),
 ('dunes', u'NNS'),
 ('there', u'EX'),
 ("'s", u'VBZ'),
 ('a', u'DT'),
 ('lot', u'NN'),
 ('to', u'TO'),
 ('say', u'VB'),
 ('about', u'IN'),
 ('this', u'DT'),
 ('place', u'NN'),
 ('It', u'PRP'),
 ("'s", u'VBZ'),
 ('not', u'RB'),
 ('great', u'JJ'),
 ('by', u'IN'),
 ('any', u'DT'),
 ('means', u'NNS'),
 ('The', u'DT'),
 ('staff', u'NN'),
 ('for', u'IN'),
 ('the', u'DT'),
 ('most', u'JJS'),
 ('part', u'NN'),
 ('is', u'VBZ'),
 ('always', u'RB'),
 ('pretty', u'RB'),
 ('good', u'JJ'),
 ('They', u'PRP'),
 ("'re", u'VBP'),
 ('is', u'VBZ'),
 ('a', u'DT'),
 ('high', u'JJ'),
 ('turn', u'NN'),
 ('around', u'RP'),
 ('on', u'IN'),
 ('the', u'DT'),
 ('bar', u'NN'),
 ('tenders', u'NNS'),
 ('however', u'RB'),
 ('That', u'DT'),
 ("'s", u'VBZ'),
 ('probably', u'RB'),
 ('because', u'IN'),
 ('this', u'DT'),
 ('place', u'NN'),
 ('is', u'VBZ'),
 ('always', u'RB'),
 ('dead', u'JJ'),
 ('The', u'DT'),
 ('outside', u'NN'),
 ('of', u'IN'),
 ('the', u'DT'),
 ('dunes', u'NNS'),
 ('is', u'VBZ'),
 ('Go

In [17]:
# Noun phrase extraction
dunes.noun_phrases

WordList([u'high turn', 'god', u'owners need', u'need work', u'ghetto nudie', u'cheap beer prices', u"bartender 's", 'karin', u'dj jeromy', u'cheap beer', 'im', u'dunes need', 'recognize', u"bartender 's work", u'owners need', u'nice place', 'fix', u"'ll pay", u"bartender 's", 'dj', u'cheap beer'])

The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

In [18]:
# Sentiment analysis
testimonial = TextBlob(str(sample_all.loc[:, 'review_text'][100]))
testimonial.sentiment
# Although this review seems more negative, TextBlob gave it a pretty neutral score.
# The reviewer, though talking down about the place, says he can't resist coming back anyways.
# The subjectivity is relatively high I'd say

Sentiment(polarity=0.1225111111111111, subjectivity=0.5843777777777777)

In [19]:
# Tokenization
zen = TextBlob(str(sample_all.loc[:, 'review_text'][100]))
zen.words

WordList(['The', 'dunes', 'there', "'s", 'a', 'lot', 'to', 'say', 'about', 'this', 'place', 'It', "'s", 'not', 'great', 'by', 'any', 'means', 'The', 'staff', 'for', 'the', 'most', 'part', 'is', 'always', 'pretty', 'good', 'They', "'re", 'is', 'a', 'high', 'turn', 'around', 'on', 'the', 'bar', 'tenders', 'however', 'That', "'s", 'probably', 'because', 'this', 'place', 'is', 'always', 'dead', 'The', 'outside', 'of', 'the', 'dunes', 'is', 'God', 'awful', 'and', 'the', 'owners', 'need', 'to', 'spend', 'some', 'money', 'to', 'renovate', 'this', 'dump', 'Both', 'out', 'side', 'and', 'inside', 'need', 'work', 'But', 'off', 'the', 'outside', 'of', 'the', 'building', 'I', 'nearly', 'skiped', 'this', 'place', 'It', "'s", 'looks', 'like', 'a', 'ran', 'down', 'ghetto', 'nudie', 'bar', 'that', 'you', "'re", 'most', 'likely', 'going', 'to', 'catch', 'something', 'incurable', 'The', 'only', 'two', 'reasons', 'I', 'go', 'back', 'is', 'because', 'it', "'s", 'so', 'close', 'to', 'my', 'place', 'and', 'b

In [20]:
zen.sentences

[Sentence("The dunes, there's a lot to say about this place."),
 Sentence("It's not great by any means."),
 Sentence("The staff for the most part is always pretty good."),
 Sentence("They're is a high turn around on the bar tenders however."),
 Sentence("That's probably because this place is always dead."),
 Sentence("The outside of the dunes is God awful and the owners need to spend some money to renovate this dump."),
 Sentence("Both out side and inside need work."),
 Sentence("But off the outside of the building I nearly skiped this place."),
 Sentence("It's looks like a ran down ghetto nudie bar, that you're most likely going to catch something incurable."),
 Sentence("The only two reasons I go back is because it's so close to my place and beer is cheap."),
 Sentence("Even the cheap beer prices and really close proximity to my house still make it hard to stop in some days."),
 Sentence("If it was not for bartender's like Karin and Summer, and the DJ Jeromy who is not always here, a

In [21]:
for sentence in zen.sentences:
    print(sentence.sentiment)

Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=-0.4, subjectivity=0.75)
Sentiment(polarity=0.48333333333333334, subjectivity=0.7000000000000001)
Sentiment(polarity=0.16, subjectivity=0.5399999999999999)
Sentiment(polarity=-0.2, subjectivity=0.4)
Sentiment(polarity=-0.5, subjectivity=0.525)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.05, subjectivity=0.225)
Sentiment(polarity=-0.03888888888888889, subjectivity=0.5972222222222222)
Sentiment(polarity=0.13333333333333333, subjectivity=0.5666666666666667)
Sentiment(polarity=0.1027777777777778, subjectivity=0.48055555555555546)
Sentiment(polarity=0.4, subjectivity=0.7)
Sentiment(polarity=0.5, subjectivity=0.8888888888888888)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.6, subjectivity=1.0)
Sentiment(polarity=0.5, subjectivity=0.5)
Sentiment(polarity=0.4, subjectivity=0.7)


In [36]:
# find most common words in sample_bad and sample_good
#from nltk import FreqDist
import scattertext as st
import spacy
from pprint import pprint
from sklearn.feature_extraction.text import TfidfVectorizer

bad_text = sample_bad['review_text'].tolist()

tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(bad_text)

# nlp = spacy.en.English()
# bad_corpus = st.CorpusFromPandas(sample_bad,
#                             category_col = 'category',
#                             text_col = 'review_text',
#                             nlp = nlp).build()

NameError: name 'tokenize' is not defined

In [32]:
bad_text

0      Terrible service. Ordered two dishes, never ev...
1      I don't know why I keep coming back to this pl...
2      Pani puri was great but the woman ripped us of...
3      Never again.  The apple french toast (~$12) an...
4      Unfortunately I am not able to critique the fo...
5      I have been here a few times for lunch and hav...
6      Yesterday was my first visit to Marche. I got ...
7      Dissapointing.\n\n1) If you want to create a n...
8      The lady that took our order sat on the phone ...
9      All four items our family ordered were sub par...
10     Food was great!\n\nLove the ambience.\n\nMusic...
11     I, like another reviewer, really wanted to lik...
12     First let me say that I love Mexican food and ...
13     The worst Panda Express location ever. It neve...
14     Alright, this is definitely the last time I'm ...
15     Teppan was just okay...had to wait awhile for ...
16     Oh for the love of god.  I somehow ended back ...
17     I came to this restauran

# Term Frequency - Inverse Document Frequency Experimentation

In [38]:
# HIMYM EXAMPLE - USE AS REFERENCE

from collections import defaultdict
import csv
 
episodes = defaultdict(list)
with open("./../../data/sentences.csv", "r") as sentences_file:
    reader = csv.reader(sentences_file, delimiter=',')
    reader.next()
    for row in reader:
        episodes[row[1]].append(row[4])
 
for episode_id, text in episodes.iteritems():
    episodes[episode_id] = "".join(text)
 
corpus = []
for id, episode in sorted(episodes.iteritems(), key=lambda t: int(t[0])):
    corpus.append(episode)

# LET'S START WITH OUR BAD REVIEWS DATASET HERE

In [52]:
bad_dict = sample_bad.set_index('review_id')['review_text'].to_dict()

In [55]:
for review_id, review_text in bad_dict.iteritems():
    bad_dict[review_id] = "".join(review_text)

In [61]:
corpus = []
for id, review in sorted(bad_dict.iteritems(), key=lambda t: t[0]):
    corpus.append(review)

In [62]:
corpus

["Terrible service. Ordered two dishes, never even received either of them! I was going to walk out because the food was taking so long (over an hour) and the manager THREATENED TO CALL THE COPS ON ME! Eventually the gave us some food that we didn't even actually order. They forgot about our other order too. All in all, a truly terrible experience. I would not recommend this place to anyone!",
 'I don\'t know why I keep coming back to this place. Guess its because I only like their chips and queso/salsa and I live right down the street. \n\nEVERYTIME I come here, I order "to-go" (the chips and salsa) and let me name a few thigns that I am really pissed at: \n\n1) It takes someone forever (about 3-7 minutes) just to come and take my order at the to-go desk. \n\n2) 2 girls who work there always ring up my chips & salsa for $5.95. Well tonight I went up there & a different girl was working there and she rang up chips and salsa/queso up for like $11!! I said "Um no, you must have put it in

Next it’s time to train our TF/IDF model.

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')

In [64]:
tfidf_matrix =  tf.fit_transform(corpus)
feature_names = tf.get_feature_names() 

In [65]:
len(feature_names)

123207

In [67]:
tfidf_matrix

<1000x123207 sparse matrix of type '<type 'numpy.float64'>'
	with 176161 stored elements in Compressed Sparse Row format>

In [70]:
dense = tfidf_matrix.todense()
len(dense[0].tolist()[0])

123207

In [72]:
review = dense[0].tolist()[0]
phrase_scores = [pair for pair in zip(range(0, len(review)), review) if pair[1] > 0]
len(phrase_scores)

75

In [74]:
sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:100]:
   print('{0: <20} {1}'.format(phrase, score))

actually order forgot 0.132681778589
cops eventually      0.132681778589
cops eventually gave 0.132681778589
didn actually order  0.132681778589
dishes received      0.132681778589
dishes received going 0.132681778589
eventually gave      0.132681778589
eventually gave food 0.132681778589
experience recommend 0.132681778589
experience recommend place 0.132681778589
food didn actually   0.132681778589
food taking          0.132681778589
food taking long     0.132681778589
forgot order truly   0.132681778589
gave food didn       0.132681778589
going walk food      0.132681778589
hour manager         0.132681778589
hour manager threatened 0.132681778589
long hour            0.132681778589
long hour manager    0.132681778589
manager threatened   0.132681778589
manager threatened cops 0.132681778589
order forgot         0.132681778589
order forgot order   0.132681778589
order truly          0.132681778589
order truly terrible 0.132681778589
ordered dishes       0.132681778589
ordered dishes

# GOOD REVIEWS DATASET

In [80]:
good_dict = sample_good.set_index('review_id')['review_text'].to_dict()

for review_id, review_text in bad_dict.iteritems():
    bad_dict[review_id] = "".join(review_text)
    
good_corpus = []
for id, review in sorted(good_dict.iteritems(), key=lambda t: t[0]):
    good_corpus.append(review)

In [90]:
good_corpus

["Mind you the wait time could be loooong! So bring someone who is diamond or 7 star member with you if you can. So u can skip the line. All the sea food was very fresh and tasty. I love their cooked oyster! Desserts are very yummy, love their flan!! So many choices. Service was okay, our server didn't bother to check on us much but the table next to us.",
 'The service was excellent ! Owner was very friendly ! Not to mention the food was delicious and fresh. I had the Monterey sand which  with portobello  mushrooms roasted red peppers-With pasta salad!!you know ingredients are fresh because carrots and lettuce were nice and crisp . I will be back and brining friends',
 "This is one of the best restaurant in Summerlin! \n\nThe location is weird. Let's all admit that. The office park is a weird place for a restaurant but it's an amazing place to discover. \n\nThe food is exceptional. The drinks are amazing. The wine selection is outstanding. \n\nThis is my number one pick for holidays o

In [81]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')

In [84]:
tfidf_matrix_good =  tf.fit_transform(good_corpus)
feature_names = tf.get_feature_names() 

In [85]:
len(feature_names)

102983

In [87]:
dense_good = tfidf_matrix_good.todense()
len(dense_good[0].tolist()[0])

102983

In [88]:
review_good = dense_good[0].tolist()[0]
phrase_scores_good = [pair for pair in zip(range(0, len(review_good)), review_good) if pair[1] > 0]
len(phrase_scores_good)

83

In [89]:
sorted_phrase_scores_good = sorted(phrase_scores_good, key=lambda t: t[1] * -1)
for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores_good][:100]:
   print('{0: <20} {1}'.format(phrase, score))

bother check         0.123747560993
bother check table   0.123747560993
bring diamond        0.123747560993
bring diamond star   0.123747560993
check table          0.123747560993
choices service      0.123747560993
choices service okay 0.123747560993
cooked oyster        0.123747560993
cooked oyster desserts 0.123747560993
desserts yummy       0.123747560993
desserts yummy love  0.123747560993
diamond star         0.123747560993
diamond star member  0.123747560993
didn bother          0.123747560993
didn bother check    0.123747560993
flan choices         0.123747560993
flan choices service 0.123747560993
food fresh tasty     0.123747560993
fresh tasty love     0.123747560993
line sea             0.123747560993
line sea food        0.123747560993
loooong bring        0.123747560993
loooong bring diamond 0.123747560993
love cooked          0.123747560993
love cooked oyster   0.123747560993
love flan            0.123747560993
love flan choices    0.123747560993
member skip          0.12

# Okay let's do this one more time, but this time with Las Vegas reviews only, since we will be using Vegas as an example in the presentation.

There are a ton of rows which lead to a ton of text in the reviews, so I'm just going to limit the bad and good to 5000 reviews each.

In [2]:
# let's read them directly from mysql
engine = create_engine(# insert connection here)
# Load all the bad reviews
vegas_bad = pd.read_sql_query('SELECT R.*, B.business_name, B.state, B.postal_code FROM reviews as R LEFT JOIN businesses as B ON R.business_id=B.business_id WHERE B.categories LIKE "%%Restaurant%%" AND B.state = "NV" AND R.stars <= 2 LIMIT 10000;', engine)
vegas_good = pd.read_sql_query('SELECT R.*, B.business_name, B.state, B.postal_code FROM reviews as R LEFT JOIN businesses as B ON R.business_id=B.business_id WHERE B.categories LIKE "%%Restaurant%%" AND B.state = "NV" AND R.stars >= 4 LIMIT 10000;', engine)


In [3]:
vegas_good.head()

,business_id,cool,review_date,funny,review_id,stars,review_text,review_type,useful,user_id,business_name,state,postal_code
0,lKq4Qsz13FDcAVgp49uukQ,0,2012-04-07,0,---3OXpexMp0oAg77xWfYA,5,Pizza here made my night... Good people and gr...,review,2,SnXZkRN9Yf060pNTk1HMDg,Slice of Vegas,NV,89119.0
1,R1PQEK6qvrZVC9qcWfKvDA,0,2014-09-17,0,---94vtJ_5o_nikEs6hUjg,5,One of my absolute favorite restaurants! I usu...,review,1,c2MQ_LPuvtiiKFR_-OY9pg,Braddah's Island Style,NV,89119.0
2,Y-S2LhHefBdnX8VP5Bh_JA,0,2016-05-23,0,---D6-P4MpS86LYldBfX7w,4,I don't know why this place only has 3 stars b...,review,0,mCa0WvcSCwtdfeUyhhfg8w,Citizens Kitchen and Bar,NV,89119.0
3,hihud--QRriCYZw1zZvW4g,0,2016-06-04,0,--0e6CLC1aWYdx5j22F3sw,4,"I like the food, it was really fresh. The serv...",review,0,L-IyEbLgcQISlsZFMMmy2g,Gangnam Asian BBQ Dining,NV,89169.0
4,16tWXenJ1wTqugXJU7yiZg,0,2015-12-27,0,--0FCNjk2NnZ2aUji5QYBg,5,Every time I come here the food is perfect !!!...,review,0,QqF3cU-IkgmNNRxHwKIZ2w,808 Sushi,NV,89113.0


# Good vegas reviews

In [4]:
g_dict = vegas_good.set_index('review_id')['review_text'].to_dict()

for review_id, review_text in g_dict.iteritems():
    g_dict[review_id] = "".join(review_text)
    
g_corpus = []
for id, review in sorted(g_dict.iteritems(), key=lambda t: t[0]):
    g_corpus.append(review)

In [5]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')
tfidf_matrix =  tf.fit_transform(g_corpus)
feature_names = tf.get_feature_names() 
len(feature_names)

759522

In [6]:
dense_g = tfidf_matrix.todense()
len(dense_g[0].tolist()[0])

759522

In [32]:
review_vegas_good = dense_g[518].tolist()[0]
phrase_scores_good = [pair for pair in zip(range(0, len(review_vegas_good)), review_vegas_good) if pair[1] > 0]
len(phrase_scores_good)

142

In [25]:
sorted_phrase_scores_good

[(322619, 0.10236723808046848),
 (712916, 0.08204718518455649),
 (587356, 0.08142543836437673),
 (83519, 0.0804809861392202),
 (529704, 0.07622124615047483),
 (181737, 0.07506483872146211),
 (5461, 0.07188561583279197),
 (5820, 0.07188561583279197),
 (5821, 0.07188561583279197),
 (6176, 0.07188561583279197),
 (6177, 0.07188561583279197),
 (7558, 0.07188561583279197),
 (7559, 0.07188561583279197),
 (9377, 0.07188561583279197),
 (9378, 0.07188561583279197),
 (19926, 0.07188561583279197),
 (55276, 0.07188561583279197),
 (55277, 0.07188561583279197),
 (57798, 0.07188561583279197),
 (83312, 0.07188561583279197),
 (83313, 0.07188561583279197),
 (83618, 0.07188561583279197),
 (83619, 0.07188561583279197),
 (83771, 0.07188561583279197),
 (105957, 0.07188561583279197),
 (105958, 0.07188561583279197),
 (127737, 0.07188561583279197),
 (127738, 0.07188561583279197),
 (127943, 0.07188561583279197),
 (129269, 0.07188561583279197),
 (171586, 0.07188561583279197),
 (171587, 0.07188561583279197),
 (179

In [33]:
sorted_phrase_scores_good = sorted(phrase_scores_good, key=lambda t: t[1] * -1)
for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores_good][:20]:
   print('{0: <25} {1}'.format(phrase, score))

dinner                    0.10037051684
24 95                     0.0961195219683
24 95 person              0.0961195219683
95 person think           0.0961195219683
based dinner              0.0961195219683
based dinner cost         0.0961195219683
casual louder             0.0961195219683
casual louder laid        0.0961195219683
cheescake sad             0.0961195219683
cheescake sad pepsi       0.0961195219683
choices sushi             0.0961195219683
choices sushi pizza       0.0961195219683
coke oh                   0.0961195219683
coke oh pretty            0.0961195219683
cost vegas seen           0.0961195219683
crab legs egg             0.0961195219683
crowd nice                0.0961195219683
crowd nice steakhouse     0.0961195219683
delicious guess kind      0.0961195219683
different crowd           0.0961195219683


# Bad Vegas Reviews

In [34]:
b_dict = vegas_bad.set_index('review_id')['review_text'].to_dict()

for review_id, review_text in b_dict.iteritems():
    b_dict[review_id] = "".join(review_text)
    
b_corpus = []
for id, review in sorted(b_dict.iteritems(), key=lambda t: t[0]):
    b_corpus.append(review)

In [35]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')
tfidf_matrix =  tf.fit_transform(b_corpus)
feature_names = tf.get_feature_names() 
len(feature_names)

924875

In [36]:
dense_b = tfidf_matrix.todense()
len(dense_b[0].tolist()[0])

924875

In [40]:
review_vegas_bad = dense_b[372].tolist()[0]
phrase_scores_bad = [pair for pair in zip(range(0, len(review_vegas_bad)), review_vegas_bad) if pair[1] > 0]
len(phrase_scores_bad)

335

In [41]:
sorted_phrase_scores_bad = sorted(phrase_scores_bad, key=lambda t: t[1] * -1)
for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores_bad][:100]:
   print('{0: <25} {1}'.format(phrase, score))

chocolate                 0.165602787504
eating chocolate          0.120456907062
like 30                   0.104601143295
courses                   0.0804999047008
biggest                   0.073610878557
waiter                    0.0688327741437
got                       0.0634142449148
basically                 0.0611384202875
00 courses                0.0602284535312
00 courses eating         0.0602284535312
170 00                    0.0602284535312
170 00 courses            0.0602284535312
20 minutes sodas          0.0602284535312
30 minutes cheese         0.0602284535312
40 minutes asked          0.0602284535312
acted like biggest        0.0602284535312
advance traveling         0.0602284535312
advance traveling company 0.0602284535312
annoyance night           0.0602284535312
annoyance night right     0.0602284535312
ask random person         0.0602284535312
asked check soon          0.0602284535312
asked okay daughter       0.0602284535312
bad sorry                 0.0602284535

In [42]:
tfidf_matrix

<10000x924875 sparse matrix of type '<type 'numpy.float64'>'
	with 1629409 stored elements in Compressed Sparse Row format>